In [1]:
import numpy as np
from keras import backend as K
from keras.layers import Layer, Input
from skimage.feature import greycomatrix
import tensorflow as tf

class GLCM(Layer):


    def __init__(self, BATCH_SIZE = 100, distance = 1, angle = 0, levels = 2, symmetric = True, normed = True, **kwargs ):
        super(GLCM, self).__init__(**kwargs)
        self.distance = distance
        self.angle = angle
        self.levels = levels
        self.symmetric = symmetric
        self.normed = normed
        self.batch_size = BATCH_SIZE

    def compute_output_shape(self, input_shape):

        return (self.batch_size, 4)

    def binarization(self, x):
        bi_x = K.sign(x)
        return bi_x
    
    def call(self, inputs):
        #make input becomes a binarized image
        image = self.binarization(inputs)

        #calculate GLCM(in a batch shape)
        rows = image.shape[1].value
        cols = image.shape[2].value
                
        row = int(round(np.sin(self.angle))) * self.distance
        col = int(round(np.cos(self.angle))) * self.distance
        if col > 0:
            a = image[:, :rows-row, :cols-col, :]
            b = image[:, row:, col:, :]
        else:
            a = image[:, :rows-row, -col:, :]
            b = image[:, row:, :cols+col, :]

        a_or_b = K.maximum(a, b)
        a_and_b = a*b

        one = tf.ones(shape = a.shape)

        not_a = one-a
        not_b = one-b

        # not(a or b)
        M0_0 = K.map_fn(K.sum, (one-a_or_b))

        # b and (not a)
        M0_1 = K.map_fn(K.sum, (b*(not_a)))

        # a and (not b)
        M1_0 = K.map_fn(K.sum, (a*(not_b)))

        # a and b
        M1_1 = K.map_fn(K.sum, (a_and_b))
        
        if self.symmetric:
            temp = tf.stack([M0_0*2, M0_1+M1_0, M0_1+M1_0, M1_1*2], axis = 1)
            if self.normed:
                output = temp / K.map_fn(K.sum, (temp))
            else:
                output = temp
        else:
            temp = tf.stack([M0_0, M0_1, M1_0, M1_1], axis = 1)
            if self.normed:
                output = temp / K.map_fn(K.sum, (temp))
            else:
                output = temp

        
        #print("output shape: ", output.shape)
        
        return output

Using TensorFlow backend.


In [2]:
x_train = np.random.randint(2, size=(1, 28, 28, 1))

In [3]:
x = Input(batch_shape=(1, 28, 28, 1))
y = GLCM(distance=1, angle=3*np.pi/4, symmetric=False, normed=False)(x)

In [4]:
result = K.function([x], [y])([x_train])
result

[array([[ 171.,  179.,  179.,  200.]], dtype=float32)]

In [5]:
result = greycomatrix(x_train.reshape([28, 28]),
                      [1, 2, 3],
                      [0, np.pi/4, np.pi/2, 3*np.pi/4],
                      levels = 2,
                      symmetric = False,
                      normed = False)
result

array([[[[179, 178, 172, 171],
         [179, 178, 170, 171],
         [184, 162, 167, 162]],

        [[188, 174, 193, 179],
         [174, 174, 183, 179],
         [160, 166, 169, 168]]],


       [[[186, 169, 190, 179],
         [177, 169, 180, 179],
         [157, 164, 165, 164]],

        [[203, 208, 201, 200],
         [198, 208, 195, 200],
         [199, 184, 199, 182]]]], dtype=uint32)